In [24]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import networkx as nx
from collections import defaultdict
import math
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr
%matplotlib notebook
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Category20
output_notebook()

Loading BokehJS ...

In [25]:
# df1 = pd.read_csv("KAG 2016-17/Agriculture/Agriculture_KAG_2016_17.csv")

In [26]:
df=pd.read_csv("MaizeCaseFile_3.csv")

In [27]:
# df1["DCCBank_AgricultureLoan"]
# df = pd.concat([df, df1["DCCBank_AgricultureLoan"]], axis=1)

In [28]:
df

,District,KGISDist 1,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Maize_Production,DCCBank_AgricultureLoan
0,BAGALKOT,Bagalkot,23310,3450.06,840.78,4.45,2879,923.83
1,BALLARI,Ballari,9630,5269.97,586.45,67.36,21292,5020.25
2,BELAGAVI,Belagavi,7472,9921.94,413.86,260.15,10450,20868.05
3,BENGALURU,Bengaluru (Urban),35884,105785.02,853.14,5004.00,271975,17273.65
4,BENGALURU(R),Bengaluru (Rural),99644,80725.12,1081.95,6124.60,593544,24202.58
5,BIDAR,Bidar,22190,70194.91,1715.02,51.50,1759,5775.47
6,CHAMARAJANAGAR,Chamarajanagara,28357,48782.34,1827.19,3538.15,105491,8939.40
7,CHIKKABALLAPURA,Chikkaballapura,56609,69036.55,1844.32,3814.02,226532,31135.21
8,CHIKKAMAGALURU,Chikkamagaluru,38148,104566.89,1802.15,1639.85,66868,41077.53
9,CHITRADURGA,Chitradurga,67863,32999.48,1322.43,2923.79,61353,45035.20


In [29]:
# df = df.drop(10)

In [30]:
# df['RRB_AgricultureLoan'] = df['RRB_AgricultureLoan'].str.replace(',', '').astype(float)

In [31]:
# new_entry = {'District': 'Vijayanagara', 
#              'KGISDist 1': 'Vijayanagara', 
#              'TotalNPK': 9629,
#              'RRB_AgricultureLoan': 5269.97,
#              'KSCARD_PLDBank_AgricultureLoan': 586.45,
#              'SowingSeedsDistributed_Maize': 67.36,
#              'Maize_Production': 21291,
#              'DCCBank_AgricultureLoan': 5020.25 }

# new_row = pd.DataFrame([new_entry])
# df.loc[df['District'] == 'BALLARI', 'DCCBank_AgricultureLoan'] = 5020.25
# df = df.append(new_row, ignore_index=True)

In [32]:
# df.to_csv("MaizeCaseFile_3.csv", index=False)

In [33]:
df1=pd.read_csv("KAG 2016-17/Agriculture/Agriculture_KAG_2016_17.csv")

In [34]:
df.columns

Index(['District', 'KGISDist 1', 'TotalNPK', 'RRB_AgricultureLoan',
       'KSCARD_PLDBank_AgricultureLoan', 'SowingSeedsDistributed_Maize',
       'Maize_Production', 'DCCBank_AgricultureLoan'],
      dtype='object')

In [35]:
x_col_indices = [2, 3, 4, 5, 7]
x_col = df.columns[x_col_indices]
x_col

Index(['TotalNPK', 'RRB_AgricultureLoan', 'KSCARD_PLDBank_AgricultureLoan',
       'SowingSeedsDistributed_Maize', 'DCCBank_AgricultureLoan'],
      dtype='object')

In [36]:
# x_col_indices = [2, 5, 3, 4, 7]  # Column indices are 0-based

# # Get the column names corresponding to the indices
# x_col = df.columns[x_col_indices]

# # Concatenate the columns as per your requirement
# x_col_combined = list(x_col[:2]) + [df.columns[3] + "_" + df.columns[4] + "_" + df.columns[7]]

# # Display the resulting column names
# print(x_col_combined)


In [37]:
# x_col_indices = [2, 3, 4]
# x_col = df.columns[x_col_indices]
# x=df[x_col]
# y=df['Maize_Production']
# x=sm.add_constant(x)
# model=sm.OLS(y,x).fit()
# model.summary()

In [38]:
# x_col_indices = [2, 3, 5]
# x_col = df.columns[x_col_indices]
# x=df[x_col]
# y=df['Maize_Production']
# x=sm.add_constant(x)
# model=sm.OLS(y,x).fit()
# model.summary()

In [39]:
# x_col_indices = [2, 4, 5]
# x_col = df.columns[x_col_indices]
# x=df[x_col]
# y=df['Maize_Production']
# x=sm.add_constant(x)
# model=sm.OLS(y,x).fit()
# model.summary()

In [40]:
# x_col_indices = [3, 4, 5]
# x_col = df.columns[x_col_indices]
# x=df[x_col]
# y=df['Maize_Production']
# x=sm.add_constant(x)
# model=sm.OLS(y,x).fit()
# model.summary()

In [41]:
x=df[x_col]
x.head()

,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,DCCBank_AgricultureLoan
0,23310,3450.06,840.78,4.45,923.83
1,9630,5269.97,586.45,67.36,5020.25
2,7472,9921.94,413.86,260.15,20868.05
3,35884,105785.02,853.14,5004.00,17273.65
4,99644,80725.12,1081.95,6124.60,24202.58


In [42]:
df['Total_Argicultural_Loan'] = df.iloc[:, 3] + df.iloc[:, 4] + df.iloc[:, 7]
df

,District,KGISDist 1,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Maize_Production,DCCBank_AgricultureLoan,Total_Argicultural_Loan
0,BAGALKOT,Bagalkot,23310,3450.06,840.78,4.45,2879,923.83,5214.67
1,BALLARI,Ballari,9630,5269.97,586.45,67.36,21292,5020.25,10876.67
2,BELAGAVI,Belagavi,7472,9921.94,413.86,260.15,10450,20868.05,31203.85
3,BENGALURU,Bengaluru (Urban),35884,105785.02,853.14,5004.00,271975,17273.65,123911.81
4,BENGALURU(R),Bengaluru (Rural),99644,80725.12,1081.95,6124.60,593544,24202.58,106009.65
5,BIDAR,Bidar,22190,70194.91,1715.02,51.50,1759,5775.47,77685.40
6,CHAMARAJANAGAR,Chamarajanagara,28357,48782.34,1827.19,3538.15,105491,8939.40,59548.93
7,CHIKKABALLAPURA,Chikkaballapura,56609,69036.55,1844.32,3814.02,226532,31135.21,102016.08
8,CHIKKAMAGALURU,Chikkamagaluru,38148,104566.89,1802.15,1639.85,66868,41077.53,147446.57
9,CHITRADURGA,Chitradurga,67863,32999.48,1322.43,2923.79,61353,45035.20,79357.11


In [43]:
x_col_indices = [2, 5, 8] 
x_col = df.columns[x_col_indices]

In [44]:
x_col

Index(['TotalNPK', 'SowingSeedsDistributed_Maize', 'Total_Argicultural_Loan'], dtype='object')

In [45]:
x = df[x_col]
x.head()

,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan
0,23310,4.45,5214.67
1,9630,67.36,10876.67
2,7472,260.15,31203.85
3,35884,5004.00,123911.81
4,99644,6124.60,106009.65


In [46]:
x.corr()

,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan
TotalNPK,1.000000,0.603043,0.732918
SowingSeedsDistributed_Maize,0.603043,1.000000,0.537591
Total_Argicultural_Loan,0.732918,0.537591,1.000000


In [47]:
p_values = x.corr(method=lambda i, j: pearsonr(i, j)[1])  # Calculate p-values
p_values

,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan
TotalNPK,1.000000,0.000330,0.000003
SowingSeedsDistributed_Maize,0.000330,1.000000,0.001815
Total_Argicultural_Loan,0.000003,0.001815,1.000000


In [48]:
checker = df[df.columns[[2,5,6,8]]]
p_values = checker.corr(method=lambda i, j: pearsonr(i, j)[1])
p_values

,TotalNPK,SowingSeedsDistributed_Maize,Maize_Production,Total_Argicultural_Loan
TotalNPK,1.000000,3.297850e-04,1.950609e-04,0.000003
SowingSeedsDistributed_Maize,0.000330,1.000000e+00,6.993347e-12,0.001815
Maize_Production,0.000195,6.993347e-12,1.000000e+00,0.001741
Total_Argicultural_Loan,0.000003,1.815462e-03,1.740595e-03,1.000000


In [49]:
checker.corr()

,TotalNPK,SowingSeedsDistributed_Maize,Maize_Production,Total_Argicultural_Loan
TotalNPK,1.000000,0.603043,0.620680,0.732918
SowingSeedsDistributed_Maize,0.603043,1.000000,0.898436,0.537591
Maize_Production,0.620680,0.898436,1.000000,0.539380
Total_Argicultural_Loan,0.732918,0.537591,0.539380,1.000000


In [50]:
y=df['Maize_Production']
y.head()

0      2879
1     21292
2     10450
3    271975
4    593544
Name: Maize_Production, dtype: int64

In [51]:
x.corr()

,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan
TotalNPK,1.000000,0.603043,0.732918
SowingSeedsDistributed_Maize,0.603043,1.000000,0.537591
Total_Argicultural_Loan,0.732918,0.537591,1.000000


In [52]:
x=sm.add_constant(x)
x.head()

,const,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan
0,1.0,23310,4.45,5214.67
1,1.0,9630,67.36,10876.67
2,1.0,7472,260.15,31203.85
3,1.0,35884,5004.00,123911.81
4,1.0,99644,6124.60,106009.65


In [53]:
# x['constant_100'] = 1000
# x = x[['constant_100'] + [col for col in x.columns if col != 'constant_100']]
# x = sm.add_constant(x, prepend=False)
# x.head()

In [54]:
model=sm.OLS(y,x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Maize_Production   R-squared:                       0.817
Model:                            OLS   Adj. R-squared:                  0.797
Method:                 Least Squares   F-statistic:                     40.19
Date:                Wed, 27 Mar 2024   Prob (F-statistic):           4.27e-10
Time:                        13:17:39   Log-Likelihood:                -389.51
No. Observations:                  31   AIC:                             787.0
Df Residuals:                      27   BIC:                             792.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                        -8411.7126   2.45e+04     -0.343      0.734   -5.87e+04    4.19e+04
TotalNPK                         0.4788      0.537      0.891      0.381      -0.624       1.581
SowingSeedsDistributed_Maize    40.5784      5.177      7.839      0.000      29.957      51.200
Total_Argicultural_Loan          0.0312      0.301      0.104      0.918      -0.586       0.648
==============================================================================
Omnibus:                       39.287   Durbin-Watson:                   1.607
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              141.914
Skew:                           2.607   Prob(JB):                     1.53e-31
Kurtosis:                      12.093   Cond. No.                     2.46e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.46e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [55]:
y.mean()

134853.12903225806

In [56]:
# target=10

In [57]:
model.params.values

array([-8.41171264e+03,  4.78763053e-01,  4.05783822e+01,  3.12171342e-02])

In [58]:
coef=model.params.values[1:]
# coef

In [59]:
coef

array([4.78763053e-01, 4.05783822e+01, 3.12171342e-02])

In [60]:
# target=5
# target

In [61]:
x.drop('const',axis=1,inplace=True)

In [62]:
x.head()

,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan
0,23310,4.45,5214.67
1,9630,67.36,10876.67
2,7472,260.15,31203.85
3,35884,5004.00,123911.81
4,99644,6124.60,106009.65


In [63]:
import numpy as np
import pandas as pd
from itertools import product

arr = np.arange(0, 1.1, 0.1)
columns=['Sensitivity (TotalNPK)', 'Sensitivity (SowingSeedsDistributed_Maize)' , 'Sensitivity (Total_Argicultural_Loan)']
# Generate all possible combinations of elements in arr
combinations = product(arr, repeat=len(columns))

# alpha_df = pd.DataFrame(combinations, columns=['Sensitivity (TotalNPK)', 'Sensitivity (RRB_AgricultureLoan)', 'Sensitivity (KSCARD_PLDBank_AgricultureLoan)', 'Sensitivity (SowingSeedsDistributed_Maize)'])
alpha_df = pd.DataFrame(combinations, columns=columns)

In [64]:
alpha_df.head()

,Sensitivity (TotalNPK),Sensitivity (SowingSeedsDistributed_Maize),Sensitivity (Total_Argicultural_Loan)
0,0.0,0.0,0.0
1,0.0,0.0,0.1
2,0.0,0.0,0.2
3,0.0,0.0,0.3
4,0.0,0.0,0.4


In [65]:
import pandas as pd
import numpy as np

x_new_all_comb = pd.DataFrame()
target=150000

def computing_new_x(alpha_x):

    temp_df = (x - np.array(alpha_x) * (y.mean() - target) / coef)
    alpha = pd.DataFrame(np.tile(alpha_x, (len(temp_df), 1)),columns=alpha_x.index)
    temp_df = pd.concat([temp_df, alpha], axis=1)
    return temp_df

for index, row in alpha_df.iterrows():
    x_new_all_comb = pd.concat([x_new_all_comb, computing_new_x(row)], ignore_index=True)

In [66]:
x_new_all_comb

,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan,Sensitivity (TotalNPK),Sensitivity (SowingSeedsDistributed_Maize),Sensitivity (Total_Argicultural_Loan)
0,23310.000000,4.450000,5214.670000,0.0,0.0,0.0
1,9630.000000,67.360000,10876.670000,0.0,0.0,0.0
2,7472.000000,260.150000,31203.850000,0.0,0.0,0.0
3,35884.000000,5004.000000,123911.810000,0.0,0.0,0.0
4,99644.000000,6124.600000,106009.650000,0.0,0.0,0.0
...,...,...,...,...,...,...
41256,85636.510191,538.274393,634007.868534,1.0,1.0,1.0
41257,123026.510191,373.274393,537048.438534,1.0,1.0,1.0
41258,148872.510191,387.624393,633844.078534,1.0,1.0,1.0
41259,105189.510191,5665.274393,556703.988534,1.0,1.0,1.0


In [67]:
x_new_all_comb[:31]

,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan,Sensitivity (TotalNPK),Sensitivity (SowingSeedsDistributed_Maize),Sensitivity (Total_Argicultural_Loan)
0,23310.0,4.45,5214.67,0.0,0.0,0.0
1,9630.0,67.36,10876.67,0.0,0.0,0.0
2,7472.0,260.15,31203.85,0.0,0.0,0.0
3,35884.0,5004.00,123911.81,0.0,0.0,0.0
4,99644.0,6124.60,106009.65,0.0,0.0,0.0
5,22190.0,51.50,77685.40,0.0,0.0,0.0
6,28357.0,3538.15,59548.93,0.0,0.0,0.0
7,56609.0,3814.02,102016.08,0.0,0.0,0.0
8,38148.0,1639.85,147446.57,0.0,0.0,0.0
9,67863.0,2923.79,79357.11,0.0,0.0,0.0


# Neg values

In [68]:
def clip(col):
    return np.maximum(col, 0)

x_new_all_comb = x_new_all_comb.apply(clip)
x_new_all_comb

,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan,Sensitivity (TotalNPK),Sensitivity (SowingSeedsDistributed_Maize),Sensitivity (Total_Argicultural_Loan)
0,23310.000000,4.450000,5214.670000,0.0,0.0,0.0
1,9630.000000,67.360000,10876.670000,0.0,0.0,0.0
2,7472.000000,260.150000,31203.850000,0.0,0.0,0.0
3,35884.000000,5004.000000,123911.810000,0.0,0.0,0.0
4,99644.000000,6124.600000,106009.650000,0.0,0.0,0.0
...,...,...,...,...,...,...
41256,85636.510191,538.274393,634007.868534,1.0,1.0,1.0
41257,123026.510191,373.274393,537048.438534,1.0,1.0,1.0
41258,148872.510191,387.624393,633844.078534,1.0,1.0,1.0
41259,105189.510191,5665.274393,556703.988534,1.0,1.0,1.0


In [69]:
x_new_all_comb=sm.add_constant(x_new_all_comb)
x_new_all_comb

,const,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan,Sensitivity (TotalNPK),Sensitivity (SowingSeedsDistributed_Maize),Sensitivity (Total_Argicultural_Loan)
0,1.0,23310.000000,4.450000,5214.670000,0.0,0.0,0.0
1,1.0,9630.000000,67.360000,10876.670000,0.0,0.0,0.0
2,1.0,7472.000000,260.150000,31203.850000,0.0,0.0,0.0
3,1.0,35884.000000,5004.000000,123911.810000,0.0,0.0,0.0
4,1.0,99644.000000,6124.600000,106009.650000,0.0,0.0,0.0
...,...,...,...,...,...,...,...
41256,1.0,85636.510191,538.274393,634007.868534,1.0,1.0,1.0
41257,1.0,123026.510191,373.274393,537048.438534,1.0,1.0,1.0
41258,1.0,148872.510191,387.624393,633844.078534,1.0,1.0,1.0
41259,1.0,105189.510191,5665.274393,556703.988534,1.0,1.0,1.0


In [70]:
y_new_all_comb=sm.add_constant(x_new_all_comb.iloc[:,:len(columns)+1]).dot(model.params.values)
y_new_all_comb

0          3091.614978
1          -728.326149
2          6696.165792
3        215690.616841
4        291129.830049
             ...      
41256     74222.097197
41257     82400.819144
41258     98378.911283
41259    289215.511064
41260     44711.807991
Length: 41261, dtype: float64

In [71]:
model.params.values

array([-8.41171264e+03,  4.78763053e-01,  4.05783822e+01,  3.12171342e-02])

In [72]:
x_new_all_comb.iloc[1,:len(columns)+1]

const                               1.00
TotalNPK                         9630.00
SowingSeedsDistributed_Maize       67.36
Total_Argicultural_Loan         10876.67
Name: 1, dtype: float64

In [73]:
y_new_all_comb[:31]

0       3091.614978
1       -728.326149
2       6696.165792
3     215690.616841
4     291129.830049
5       6726.941748
6     150595.921141
7     176641.995932
8      80997.459708
9     145198.554044
10      3036.517639
11     -4596.020089
12    389943.934661
13     22778.413308
14     55798.056630
15     88293.098378
16     87280.109231
17    568948.555353
18    133243.645677
19    269493.585738
20    112217.739460
21    172125.875837
22    485504.297119
23     60775.699465
24    289427.385418
25      8409.249931
26     28781.484294
27     36960.206240
28     52938.298380
29    243774.898160
30      -728.804912
dtype: float64

In [74]:
y_new_all_comb=np.maximum(y_new_all_comb,0)
y_new_all_comb

0          3091.614978
1             0.000000
2          6696.165792
3        215690.616841
4        291129.830049
             ...      
41256     74222.097197
41257     82400.819144
41258     98378.911283
41259    289215.511064
41260     44711.807991
Length: 41261, dtype: float64

In [75]:
def init_graph(G,node_adj_frame):
    G.add_nodes_from([i for i in range(len(node_adj_frame))])
    labels = {}
    labels = node_adj_frame.columns
    for i in range(len(node_adj_frame)):
        snode = node_adj_frame[labels[0]][i]-1
        temp = node_adj_frame[labels[2]][i]
        if ',' in str(temp):
            sedge_arr = temp.split(',')
            for j in range(0, len(sedge_arr)):
                k = int(sedge_arr[j])
                G.add_edge(snode, k-1)
        elif np.isnan(temp):
            print("ERROR: Not found in the adjacency excel sheet")
        else:
            G.add_edge(snode, int(temp)-1)
    return

def init_graph_attr(G, AdjFile, values):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["District_Name"].tolist()
    node_list.insert(80, "")
    nodeAttr = {}
    init_graph(G, node_adj_frame)
    
    values = values.fillna(0)
    capability_vector = list(zip(*[values]))
    
    node_attri_dict = dict(zip(node_list[:31], capability_vector))
    node_attri_dict = dict((k, v) for k, v in node_attri_dict.items())

    for i in range(len(node_adj_frame)):
        temp = {}
        temp["capabilityvector"] = node_attri_dict[node_list[i]]
        temp["nodeStress"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp

    nx.set_node_attributes(G, nodeAttr)

def addList(l1,l2):
    for i in range(len(l1)):
        l1[i] = l1[i] + l2[i]
    return l1
def divList(l1,k):
    for i in range(len(l1)):
        l1[i] = l1[i]/k
    return l1

def l2_normalization(l1,l2):
    k = 0
    for i in range(len(l1)):
        k+= (l1[i] - l2[i])**2
    return math.sqrt(k)

def get_node_stress(G,dim):
    stress_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return stress_dict

def get_node_stability(G,dim):
    satbility_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStability"] = 1 - l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            satbility_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStability"]
        except(KeyError):
            pass
    return satbility_dict

In [76]:
x_new_all_comb['Maize_Production']=y_new_all_comb

In [77]:
x_new_all_comb.drop("const",axis=1,inplace=True)
x_new_all_comb

,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan,Sensitivity (TotalNPK),Sensitivity (SowingSeedsDistributed_Maize),Sensitivity (Total_Argicultural_Loan),Maize_Production
0,23310.000000,4.450000,5214.670000,0.0,0.0,0.0,3091.614978
1,9630.000000,67.360000,10876.670000,0.0,0.0,0.0,0.000000
2,7472.000000,260.150000,31203.850000,0.0,0.0,0.0,6696.165792
3,35884.000000,5004.000000,123911.810000,0.0,0.0,0.0,215690.616841
4,99644.000000,6124.600000,106009.650000,0.0,0.0,0.0,291129.830049
...,...,...,...,...,...,...,...
41256,85636.510191,538.274393,634007.868534,1.0,1.0,1.0,74222.097197
41257,123026.510191,373.274393,537048.438534,1.0,1.0,1.0,82400.819144
41258,148872.510191,387.624393,633844.078534,1.0,1.0,1.0,98378.911283
41259,105189.510191,5665.274393,556703.988534,1.0,1.0,1.0,289215.511064


In [78]:
x_new_all_comb['Maize_Production'].mean()

157574.04663273954

In [79]:
dim = 1
G = nx.Graph()
adjacency_file = "Karnataka_District_Adjacency_File - Copy.xlsx"

column = df["Maize_Production"]
column_values = column.values.reshape(-1, 1)
scaler = MinMaxScaler()
normalized_column_values = scaler.fit_transform(column_values)

column_to_pass = pd.Series(normalized_column_values.flatten(), name="Maize_Production")
init_graph_attr(G,adjacency_file , column_to_pass )

initialstability= get_node_stability(G,dim)
df["Initial Stability"] = df["District"].map(initialstability)

In [80]:
df.head()

,District,KGISDist 1,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Maize_Production,DCCBank_AgricultureLoan,Total_Argicultural_Loan,Initial Stability
0,BAGALKOT,Bagalkot,23310,3450.06,840.78,4.45,2879,923.83,5214.67,0.957557
1,BALLARI,Ballari,9630,5269.97,586.45,67.36,21292,5020.25,10876.67,0.958333
2,BELAGAVI,Belagavi,7472,9921.94,413.86,260.15,10450,20868.05,31203.85,0.274673
3,BENGALURU,Bengaluru (Urban),35884,105785.02,853.14,5004.00,271975,17273.65,123911.81,0.986036
4,BENGALURU(R),Bengaluru (Rural),99644,80725.12,1081.95,6124.60,593544,24202.58,106009.65,0.972711


In [81]:
df.to_excel("MaizeCaseFile_2.xlsx",index=False)

In [82]:
df

,District,KGISDist 1,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Maize_Production,DCCBank_AgricultureLoan,Total_Argicultural_Loan,Initial Stability
0,BAGALKOT,Bagalkot,23310,3450.06,840.78,4.45,2879,923.83,5214.67,0.957557
1,BALLARI,Ballari,9630,5269.97,586.45,67.36,21292,5020.25,10876.67,0.958333
2,BELAGAVI,Belagavi,7472,9921.94,413.86,260.15,10450,20868.05,31203.85,0.274673
3,BENGALURU,Bengaluru (Urban),35884,105785.02,853.14,5004.00,271975,17273.65,123911.81,0.986036
4,BENGALURU(R),Bengaluru (Rural),99644,80725.12,1081.95,6124.60,593544,24202.58,106009.65,0.972711
5,BIDAR,Bidar,22190,70194.91,1715.02,51.50,1759,5775.47,77685.40,0.980069
6,CHAMARAJANAGAR,Chamarajanagara,28357,48782.34,1827.19,3538.15,105491,8939.40,59548.93,0.844624
7,CHIKKABALLAPURA,Chikkaballapura,56609,69036.55,1844.32,3814.02,226532,31135.21,102016.08,0.872768
8,CHIKKAMAGALURU,Chikkamagaluru,38148,104566.89,1802.15,1639.85,66868,41077.53,147446.57,0.744033
9,CHITRADURGA,Chitradurga,67863,32999.48,1322.43,2923.79,61353,45035.20,79357.11,0.862275


In [83]:
x_new_all_comb[0:31]

,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan,Sensitivity (TotalNPK),Sensitivity (SowingSeedsDistributed_Maize),Sensitivity (Total_Argicultural_Loan),Maize_Production
0,23310.0,4.45,5214.67,0.0,0.0,0.0,3091.614978
1,9630.0,67.36,10876.67,0.0,0.0,0.0,0.000000
2,7472.0,260.15,31203.85,0.0,0.0,0.0,6696.165792
3,35884.0,5004.00,123911.81,0.0,0.0,0.0,215690.616841
4,99644.0,6124.60,106009.65,0.0,0.0,0.0,291129.830049
5,22190.0,51.50,77685.40,0.0,0.0,0.0,6726.941748
6,28357.0,3538.15,59548.93,0.0,0.0,0.0,150595.921141
7,56609.0,3814.02,102016.08,0.0,0.0,0.0,176641.995932
8,38148.0,1639.85,147446.57,0.0,0.0,0.0,80997.459708
9,67863.0,2923.79,79357.11,0.0,0.0,0.0,145198.554044


In [84]:
def calculate_and_map_stability(G, df, adjacency_file, column_values, dim):
    G = nx.Graph()
    scaler = MinMaxScaler()
    
    normalized_column_values = scaler.fit_transform(column_values.reshape(-1, 1))
    column_to_pass = pd.Series(normalized_column_values.flatten(), name="Column")

    init_graph_attr(G, adjacency_file, column_to_pass)
    taluka_stress_dict = get_node_stability(G, dim)

    stability_column_name = "New Stability"
    stability_column = df["District"].map(taluka_stress_dict)

    return stability_column, stability_column_name

Cap_Vector = "Maize_Production"

column_values = x_new_all_comb[Cap_Vector].iloc[0:31].values
result_column, result_column_name = calculate_and_map_stability(G, df, adjacency_file, column_values, dim)

df[result_column_name] = result_column

In [85]:
def calculate_and_map_stress(G, df, adjacency_file, column_values, dim):
    G = nx.Graph()
    scaler = MinMaxScaler()
    
    normalized_column_values = scaler.fit_transform(column_values.reshape(-1, 1))
    column_to_pass = pd.Series(normalized_column_values.flatten(), name="Column")

    init_graph_attr(G, adjacency_file, column_to_pass)
    taluka_stress_dict = get_node_stress(G, dim)

    stress_column_name = "New Stress"
    stress_column = df["District"].map(taluka_stress_dict)

    return stress_column, stress_column_name

Cap_Vector = "Maize_Production"

column_values = x_new_all_comb[Cap_Vector].iloc[0:31].values 
result_column, result_column_name = calculate_and_map_stress(G, df, adjacency_file, column_values, dim)

df[result_column_name] = result_column

In [86]:
def calculate_and_map_stability(G, df, adjacency_file, column_values, dim):
    G = nx.Graph()

    scaler = MinMaxScaler()
    normalized_column_values = scaler.fit_transform(column_values.reshape(-1, 1))

    column_to_pass = pd.Series(normalized_column_values.flatten(), name="Column")

    init_graph_attr(G, adjacency_file, column_to_pass)

    stability_dict = get_node_stability(G, dim)
    stability_column = df["District"].map(stability_dict)

    return stability_column

x_new_all_comb["New Stability"] = np.nan
Cap_Vector = "Maize_Production"

chunk_size = 31
for i in range(0, len(x_new_all_comb), chunk_size):

    chunk_values = x_new_all_comb[Cap_Vector].iloc[i:i + chunk_size].values

    result_column = calculate_and_map_stability(G, df, adjacency_file, chunk_values, dim)
    
    x_new_all_comb["New Stability"].iloc[i:i + chunk_size] = result_column

x_new_all_comb["New Stability"] = pd.to_numeric(x_new_all_comb["New Stability"])

In [87]:
def calculate_and_map_stress(G, df, adjacency_file, column_values, dim):
    G = nx.Graph()

    scaler = MinMaxScaler()
    normalized_column_values = scaler.fit_transform(column_values.reshape(-1, 1))

    column_to_pass = pd.Series(normalized_column_values.flatten(), name="Column")

    init_graph_attr(G, adjacency_file, column_to_pass)

    stress_dict = get_node_stress(G, dim)
    stress_column = df["District"].map(stress_dict)

    return stress_column

x_new_all_comb["New Stress"] = np.nan
Cap_Vector = "Maize_Production"

chunk_size = 31
for i in range(0, len(x_new_all_comb), chunk_size):

    chunk_values = x_new_all_comb[Cap_Vector].iloc[i:i + chunk_size].values

    result_column = calculate_and_map_stress(G, df, adjacency_file, chunk_values, dim)
    
    x_new_all_comb["New Stress"].iloc[i:i + chunk_size] = result_column

x_new_all_comb["New Stress"] = pd.to_numeric(x_new_all_comb["New Stress"])

In [88]:
x_new_all_comb

,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan,Sensitivity (TotalNPK),Sensitivity (SowingSeedsDistributed_Maize),Sensitivity (Total_Argicultural_Loan),Maize_Production,New Stability,New Stress
0,23310.000000,4.450000,5214.670000,0.0,0.0,0.0,3091.614978,0.937978,0.062022
1,9630.000000,67.360000,10876.670000,0.0,0.0,0.0,0.000000,0.858178,0.141822
2,7472.000000,260.150000,31203.850000,0.0,0.0,0.0,6696.165792,0.262891,0.737109
3,35884.000000,5004.000000,123911.810000,0.0,0.0,0.0,215690.616841,0.997570,0.002430
4,99644.000000,6124.600000,106009.650000,0.0,0.0,0.0,291129.830049,0.912784,0.087216
...,...,...,...,...,...,...,...,...,...
41256,85636.510191,538.274393,634007.868534,1.0,1.0,1.0,74222.097197,0.823886,0.176114
41257,123026.510191,373.274393,537048.438534,1.0,1.0,1.0,82400.819144,0.639143,0.360857
41258,148872.510191,387.624393,633844.078534,1.0,1.0,1.0,98378.911283,0.898559,0.101441
41259,105189.510191,5665.274393,556703.988534,1.0,1.0,1.0,289215.511064,0.939509,0.060491


In [89]:
# x_new_all_comb.to_excel("x_col_new_all_comb_10.xlsx", index =False)

In [90]:
# x_new_all_comb = pd.read_excel('x_col_new_all_comb_10.xlsx')
# x_new_all_comb = x_new_all_comb.dropna()
# x_new_all_comb

In [91]:
# x_new_all_comb.to_excel("x_col_new_all_comb_10.xlsx", index =False)

In [92]:
district_column = pd.DataFrame({
    "KGISDist 1": df["KGISDist 1"].tolist() * (len(x_new_all_comb) // len(df)),
    "District": df["District"].tolist() * (len(x_new_all_comb) // len(df)),
    "Target Maize Production": [target] * len(x_new_all_comb)
})

district_column = pd.concat([district_column, district_column.head(len(x_new_all_comb) % len(df))])

district_column.reset_index(drop=True, inplace=True)

x_new_all_comb["KGISDist 1"] = district_column["KGISDist 1"]
x_new_all_comb["District"] = district_column["District"]
x_new_all_comb["Target Maize Production"] = district_column["Target Maize Production"]

x_new_all_comb.drop(columns=["KGISDist 1", "District", "Target Maize Production"], inplace=True)


x_new_all_comb.insert(0, "District", district_column["District"])
x_new_all_comb.insert(1, "KGISDist 1", district_column["KGISDist 1"])
x_new_all_comb.insert(11, "Target Maize Production", district_column["Target Maize Production"])

In [93]:
x_new_all_comb[0:31]

,District,KGISDist 1,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan,Sensitivity (TotalNPK),Sensitivity (SowingSeedsDistributed_Maize),Sensitivity (Total_Argicultural_Loan),Maize_Production,New Stability,New Stress,Target Maize Production
0,BAGALKOT,Bagalkot,23310.0,4.45,5214.67,0.0,0.0,0.0,3091.614978,0.937978,0.062022,150000
1,BALLARI,Ballari,9630.0,67.36,10876.67,0.0,0.0,0.0,0.000000,0.858178,0.141822,150000
2,BELAGAVI,Belagavi,7472.0,260.15,31203.85,0.0,0.0,0.0,6696.165792,0.262891,0.737109,150000
3,BENGALURU,Bengaluru (Urban),35884.0,5004.00,123911.81,0.0,0.0,0.0,215690.616841,0.997570,0.002430,150000
4,BENGALURU(R),Bengaluru (Rural),99644.0,6124.60,106009.65,0.0,0.0,0.0,291129.830049,0.912784,0.087216,150000
5,BIDAR,Bidar,22190.0,51.50,77685.40,0.0,0.0,0.0,6726.941748,0.964193,0.035807,150000
6,CHAMARAJANAGAR,Chamarajanagara,28357.0,3538.15,59548.93,0.0,0.0,0.0,150595.921141,0.934937,0.065063,150000
7,CHIKKABALLAPURA,Chikkaballapura,56609.0,3814.02,102016.08,0.0,0.0,0.0,176641.995932,0.786704,0.213296,150000
8,CHIKKAMAGALURU,Chikkamagaluru,38148.0,1639.85,147446.57,0.0,0.0,0.0,80997.459708,0.964584,0.035416,150000
9,CHITRADURGA,Chitradurga,67863.0,2923.79,79357.11,0.0,0.0,0.0,145198.554044,0.904491,0.095509,150000


In [94]:
x_new_all_comb.to_csv("Maize_Prod_150000.csv", index =False)

In [95]:
# x_new_all_comb.to_excel("Maize_Prod_150000.xlsx", index =False)

In [96]:
# x_new_all_comb = pd.read_csv('Maize_Prod_150000.csv')
df = x_new_all_comb

In [97]:
type(y)

pandas.core.series.Series

In [98]:
y

0       2879
1      21292
2      10450
3     271975
4     593544
5       1759
6     105491
7     226532
8      66868
9      61353
10         0
11       109
12    333937
13     17921
14     19387
15    137154
16    147886
17    576758
18    226327
19    251699
20     90634
21    138476
22    413487
23     24088
24    208090
25      2791
26     14621
27      2673
28       723
29    190252
30     21291
Name: Maize_Production, dtype: int64

In [99]:
df

,District,KGISDist 1,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan,Sensitivity (TotalNPK),Sensitivity (SowingSeedsDistributed_Maize),Sensitivity (Total_Argicultural_Loan),Maize_Production,New Stability,New Stress,Target Maize Production
0,BAGALKOT,Bagalkot,23310.000000,4.450000,5214.670000,0.0,0.0,0.0,3091.614978,0.937978,0.062022,150000
1,BALLARI,Ballari,9630.000000,67.360000,10876.670000,0.0,0.0,0.0,0.000000,0.858178,0.141822,150000
2,BELAGAVI,Belagavi,7472.000000,260.150000,31203.850000,0.0,0.0,0.0,6696.165792,0.262891,0.737109,150000
3,BENGALURU,Bengaluru (Urban),35884.000000,5004.000000,123911.810000,0.0,0.0,0.0,215690.616841,0.997570,0.002430,150000
4,BENGALURU(R),Bengaluru (Rural),99644.000000,6124.600000,106009.650000,0.0,0.0,0.0,291129.830049,0.912784,0.087216,150000
...,...,...,...,...,...,...,...,...,...,...,...,...
41256,UDUPI,Udupi,85636.510191,538.274393,634007.868534,1.0,1.0,1.0,74222.097197,0.823886,0.176114,150000
41257,UTTARA KANNADA,Uttara Kannada,123026.510191,373.274393,537048.438534,1.0,1.0,1.0,82400.819144,0.639143,0.360857,150000
41258,VIJAYAPURA,Vijayapura,148872.510191,387.624393,633844.078534,1.0,1.0,1.0,98378.911283,0.898559,0.101441,150000
41259,YADGIRI,Yadgir,105189.510191,5665.274393,556703.988534,1.0,1.0,1.0,289215.511064,0.939509,0.060491,150000


In [100]:
df['Percentage Change'] = df['Maize_Production'] - np.tile(y.values, len(df) // 31 + 1)[:len(df)]

for i in range(0, len(df), 31):
    df.loc[i:i+30, 'Percentage Change'] = df.loc[i:i+30, 'Percentage Change'] * 100 /  y.values
    
df['Percentage Change'] = df['Percentage Change'].replace([np.inf, -np.inf], 0)

In [101]:
df.to_csv("Maize_Prod_150000.csv", index =False)
df.to_excel("Maize_Prod_150000.xlsx", index =False)

In [102]:
# x_new_all_comb['Maternal Deaths'].mean()

In [103]:
# import pandas as pd
# import plotly.express as px
# import ipywidgets as widgets
# from ipywidgets import interact
# from IPython.display import clear_output

# # Function to update the plot based on alpha values
# def update_plot(alpha1, alpha2, alpha3, alpha4):
#     # Filter the dataset based on the selected alpha values
#     filtered_df = df[(df['Sensitivity (Health Facility)'] == alpha1) &
#                      (df['Sensitivity (Caesarean)'] == alpha2) &
#                      (df['Sensitivity (Leprosy Female)'] == alpha3) &
#                      (df['Sensitivity (TB Female)'] == alpha4)]

#     fig = px.scatter(filtered_df, x='New Stability', y='Impact', title='Stability vs Impact',
#                      labels={'Impact': 'Impact', 'New Stability': 'Stability'},
#                      text='KGISDist 1', opacity=1,
#                      color='KGISDist 1', color_discrete_sequence=['darkblue']
#                      )   # Include district names in labels

#     # Customize the layout to make text labels visible
#     fig.update_traces(textposition='top center', showlegend=False)
#     fig.update_layout(annotations=[
#         dict(
#             x=0.5,
#             y=-0.25,
#             showarrow=False,
#             text=f'Alpha 1: {alpha1}, Alpha 2: {alpha2}, Alpha 3: {alpha3}, Alpha 4: {alpha4}',
#             xref="paper",
#             yref="paper"
#         )])

#     # Add average lines
#     avg_stability = filtered_df['New Stability'].mean()
#     avg_impact = filtered_df['Impact'].mean()
    
#     fig.add_shape(
#         type='line',
#         x0=avg_stability, y0=0, x1=avg_stability, y1=1,
#         line=dict(color='red', width=2),
#         yref='paper'
#     )

#     fig.add_shape(
#         type='line',
#         x0=0, y0=avg_impact, x1=1, y1=avg_impact,
#         line=dict(color='red', width=2),
#         xref='paper'
#     )

#     # Clear the output cell to remove the old plot
#     clear_output(wait=True)
#     # Display the new plot
#     display(fig)

# # Create sliders for alphas
# alpha1_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 1:')
# alpha2_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 2:')
# alpha3_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 3:')
# alpha4_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 4:')

# # Create interactive plot
# interactive_plot = interact(update_plot, alpha1=alpha1_slider, alpha2=alpha2_slider, alpha3=alpha3_slider, alpha4=alpha4_slider)


In [104]:
# import pandas as pd
# import plotly.express as px
# import ipywidgets as widgets
# from ipywidgets import interact
# from IPython.display import clear_output

# # Function to update the plot based on alpha values
# def update_plot(alpha1, alpha2, alpha3, alpha4):
#     # Filter the dataset based on the selected alpha values
#     filtered_df = df[(df['Sensitivity (Health Facility)'] == alpha1) &
#                      (df['Sensitivity (Caesarean)'] == alpha2) &
#                      (df['Sensitivity (Leprosy Female)'] == alpha3) &
#                      (df['Sensitivity (TB Female)'] == alpha4)]

#     fig = px.scatter(filtered_df, x='New Stability', y='Impact', title='Stability vs Impact',
#                      labels={'Impact': 'Impact', 'New Stability': 'Stability'},
#                      text='KGISDist 1', opacity=1,
#                      color='KGISDist 1', color_discrete_sequence=['darkblue']
#                      )   # Include district names in labels

#     # Customize the layout to make text labels visible
#     fig.update_traces(textposition='top center', showlegend=False)
#     fig.update_layout(annotations=[
#         dict(
#             x=0.5,
#             y=-0.25,
#             showarrow=False,
#             text=f'Alpha 1: {alpha1}, Alpha 2: {alpha2}, Alpha 3: {alpha3}, Alpha 4: {alpha4}',
#             xref="paper",
#             yref="paper"
#         )])

#     # Add average lines
#     avg_stability = filtered_df['New Stability'].mean()
#     avg_impact = filtered_df['Impact'].mean()
    
#     fig.add_shape(
#         type='line',
#         x0=avg_stability, y0=0, x1=avg_stability, y1=1,
#         line=dict(color='red', width=2),
#         yref='paper'
#     )

#     fig.add_shape(
#         type='line',
#         x0=0, y0=avg_impact, x1=1, y1=avg_impact,
#         line=dict(color='red', width=2),
#         xref='paper'
#     )

#     # Clear the output cell to remove the old plot
#     clear_output(wait=True)
#     # Display the new plot
#     display(fig)

# # Create dropdowns for alphas
# alpha1_dropdown = widgets.Dropdown(options=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], value=0.5, description='Alpha 1:')
# alpha2_dropdown = widgets.Dropdown(options=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], value=0.5, description='Alpha 2:')
# alpha3_dropdown = widgets.Dropdown(options=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], value=0.5, description='Alpha 3:')
# alpha4_dropdown = widgets.Dropdown(options=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], value=0.5, description='Alpha 4:')

# # Create interactive plot
# interactive_plot = interact(update_plot, alpha1=alpha1_dropdown, alpha2=alpha2_dropdown, alpha3=alpha3_dropdown, alpha4=alpha4_dropdown)


In [105]:
abbreviation_mapping = {
    'BENGALURU': 'BLR',
    'BENGALURU(R)': 'BLR(R)',
    'RAMANAGARA': 'RGA',
    'CHITRADURGA': 'CDA',
    'DAVANAGERE': 'DVG',
    'KOLAR': 'KLR',
    'CHIKKABALLAPURA': 'CKA',
    'SHIVAMOGGA': 'SMG',
    'TUMAKURU': 'TKR',
    'CHIKKAMAGALURU': 'CMG',
    'DAKSHINA KANNADA': 'DKA',
    'UDUPI': 'UPI',
    'HASSAN': 'HSN',
    'KODAGU': 'KDG',
    'MANDYA': 'MDY',
    'MYSURU': 'MYS',
    'CHAMARAJANAGAR': 'CNR',
    'BELAGAVI': 'BLG',
    'VIJAYAPURA': 'VJP',
    'BAGALKOT': 'BKT',
    'DHARAWAD': 'DWD',
    'GADAG': 'GDG',
    'HAVERI': 'HVR',
    'UTTARA KANNADA': 'UTK',
    'BALLARI': 'BLL',
    'BIDAR': 'BDR',
    'KALABURAGI': 'KLB',
    'YADGIRI': 'YDR',
    'RAICHUR': 'RCR',
    'KOPPAL': 'KPL',
}

df['Abbreviation'] = df['District'].map(abbreviation_mapping)

In [106]:
df

,District,KGISDist 1,TotalNPK,SowingSeedsDistributed_Maize,Total_Argicultural_Loan,Sensitivity (TotalNPK),Sensitivity (SowingSeedsDistributed_Maize),Sensitivity (Total_Argicultural_Loan),Maize_Production,New Stability,New Stress,Target Maize Production,Percentage Change,Abbreviation
0,BAGALKOT,Bagalkot,23310.000000,4.450000,5214.670000,0.0,0.0,0.0,3091.614978,0.937978,0.062022,150000,7.385029,BKT
1,BALLARI,Ballari,9630.000000,67.360000,10876.670000,0.0,0.0,0.0,0.000000,0.858178,0.141822,150000,-100.000000,BLL
2,BELAGAVI,Belagavi,7472.000000,260.150000,31203.850000,0.0,0.0,0.0,6696.165792,0.262891,0.737109,150000,-35.921858,BLG
3,BENGALURU,Bengaluru (Urban),35884.000000,5004.000000,123911.810000,0.0,0.0,0.0,215690.616841,0.997570,0.002430,150000,-20.694690,BLR
4,BENGALURU(R),Bengaluru (Rural),99644.000000,6124.600000,106009.650000,0.0,0.0,0.0,291129.830049,0.912784,0.087216,150000,-50.950590,BLR(R)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41256,UDUPI,Udupi,85636.510191,538.274393,634007.868534,1.0,1.0,1.0,74222.097197,0.823886,0.176114,150000,407.640361,UPI
41257,UTTARA KANNADA,Uttara Kannada,123026.510191,373.274393,537048.438534,1.0,1.0,1.0,82400.819144,0.639143,0.360857,150000,2982.709283,UTK
41258,VIJAYAPURA,Vijayapura,148872.510191,387.624393,633844.078534,1.0,1.0,1.0,98378.911283,0.898559,0.101441,150000,13507.041671,VJP
41259,YADGIRI,Yadgir,105189.510191,5665.274393,556703.988534,1.0,1.0,1.0,289215.511064,0.939509,0.060491,150000,52.017067,YDR


In [107]:
legend_labels = [{'Abbreviation': abb, 'Full Form': full_form} 
                    for abb, full_form in zip(df['Abbreviation'].unique(), df['KGISDist 1'].unique())]

In [108]:
legend_labels

[{'Abbreviation': 'BKT', 'Full Form': 'Bagalkot'},
 {'Abbreviation': 'BLL', 'Full Form': 'Ballari'},
 {'Abbreviation': 'BLG', 'Full Form': 'Belagavi'},
 {'Abbreviation': 'BLR', 'Full Form': 'Bengaluru (Urban)'},
 {'Abbreviation': 'BLR(R)', 'Full Form': 'Bengaluru (Rural)'},
 {'Abbreviation': 'BDR', 'Full Form': 'Bidar'},
 {'Abbreviation': 'CNR', 'Full Form': 'Chamarajanagara'},
 {'Abbreviation': 'CKA', 'Full Form': 'Chikkaballapura'},
 {'Abbreviation': 'CMG', 'Full Form': 'Chikkamagaluru'},
 {'Abbreviation': 'CDA', 'Full Form': 'Chitradurga'},
 {'Abbreviation': 'DKA', 'Full Form': 'Dakshina Kannada'},
 {'Abbreviation': 'DVG', 'Full Form': 'Davanagere'},
 {'Abbreviation': 'DWD', 'Full Form': 'Dharwad'},
 {'Abbreviation': 'GDG', 'Full Form': 'Gadag'},
 {'Abbreviation': 'HSN', 'Full Form': 'Hassan'},
 {'Abbreviation': 'HVR', 'Full Form': 'Haveri'},
 {'Abbreviation': 'KLB', 'Full Form': 'Kalburgi'},
 {'Abbreviation': 'KDG', 'Full Form': 'Kodagu'},
 {'Abbreviation': 'KLR', 'Full Form': 'Kol

In [109]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import clear_output
import plotly.graph_objects as go

def update_plot(alpha1, alpha2, alpha3):
    
    filtered_df = df[(df['Sensitivity (TotalNPK)'] == alpha1) &
                     (df['Sensitivity (SowingSeedsDistributed_Maize)'] == alpha2) &
                     (df['Sensitivity (Total_Argicultural_Loan)'] == alpha3) ]

    fig = px.scatter(filtered_df, x= 'New Stability', y='Percentage Change', title='Percentage Change vs Stability',
                     labels={'Percentage Change': 'Percentage Change', 'New Stability': 'Stability'},
                     text='Abbreviation', opacity=1,color_discrete_sequence=['darkblue'])

    fig.update_traces(textposition='top center', textfont=dict(size=11))

    fig.update_layout(
        annotations=[
            dict(
                x=0.5,
                y=-0.25,
                showarrow=False,
                text=f'α<sub>NPK</sub>: {alpha1:.2f}, α<sub>MSD</sub>: {alpha2:.2f}, α<sub>TL</sub>: {alpha3:.2f} ',
                xref="paper",
                yref="paper",
                font=dict(size=13)
            )
        ],
        legend_title_text='Districts',
        legend_title=dict(font=dict(size=12)),
    )

    legend_labels = [{'Abbreviation': abb, 'Full Form': full_form} 
                     for abb, full_form in zip(filtered_df['Abbreviation'].unique(), filtered_df['KGISDist 1'].unique())]

    for label in legend_labels:
        fig.add_trace(go.Scatter(
            x=[None],
            y=[None],
            mode='markers',
            marker=dict(color='darkblue'),
            name=f"{label['Abbreviation']}: {label['Full Form']}"
        ))

    avg_stability = filtered_df['New Stability'].mean()
    avg_precent = filtered_df['Percentage Change'].mean()

    fig.add_shape(
        type='line',
        x0=avg_stability, y0=0, x1=avg_stability, y1=1,
        line=dict(color='red', width=1.5, dash='dash'),
        yref='paper'
    )

    fig.add_shape(
        type='line',
        x0=0, y0=avg_precent, x1=1, y1=avg_precent,
        line=dict(color='red', width=1.5, dash='dash'),
        xref='paper'
    )

    fig.add_trace(go.Scatter(
        x=[avg_stability],
        y=[-0.07],
        text=[f'Avg Stability: {avg_stability:.2f}'],
        mode="text",
        showlegend=False,
        textfont=dict(size=10.7)
    ))

    fig.add_trace(go.Scatter(
        x=[0.3],
        y=[avg_precent-0.04],
        text=[f'Avg Percentage Change: {avg_precent:.2f}'],
        mode="text",
        showlegend=False,
        textfont=dict(size=10.7)
    ))

    clear_output()

    display(fig)
    
alpha1_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αNPK:')
# alpha2_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αRRB:')
# alpha3_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αKS:')
alpha2_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αMSD:')
alpha3_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αTL:')

# interactive_plot = interact(update_plot, alpha1=alpha1_slider, alpha2=alpha2_slider, alpha3=alpha3_slider, alpha4=alpha4_slider, __manual=True)
# update_button = widgets.Button(description="Update Plot")
# update_button.on_click(lambda _: update_plot(alpha1_slider.value, alpha2_slider.value, alpha3_slider.value, alpha4_slider.value))
# widgets.HBox([update_button, alpha1_slider, alpha2_slider, alpha3_slider, alpha4_slider])

interactive_plot = interact(update_plot, alpha1=alpha1_slider, alpha2=alpha2_slider, alpha3=alpha3_slider, __manual=True)
update_button = widgets.Button(description="Update Plot")
update_button.on_click(lambda _: update_plot(alpha1_slider.value, alpha2_slider.value, alpha3_slider.value))
widgets.HBox([update_button, alpha1_slider, alpha2_slider, alpha3_slider])


interactive(children=(FloatSlider(value=0.5, description='αNPK:', max=1.0), FloatSlider(value=0.5, description…

In [110]:
# import pandas as pd
# import plotly.express as px
# import ipywidgets as widgets
# from ipywidgets import interact
# from IPython.display import clear_output

# # Function to update the plot based on alpha values
# def update_plot(alpha1, alpha2, alpha3, alpha4):
#     # Filter the dataset based on the selected alpha values
#     filtered_df = df[(df['Sensitivity (Health Facility)'] == alpha1) &
#                      (df['Sensitivity (Caesarean)'] == alpha2) &
#                      (df['Sensitivity (Leprosy Female)'] == alpha3) &
#                      (df['Sensitivity (TB Female)'] == alpha4)]

#     fig = px.scatter(filtered_df, x='New Stability', y='Impact', title='Stability vs Impact',
#                      labels={'Impact': 'Impact', 'New Stability': 'Stability'},
#                      text='KGISDist 1', opacity=1,
#                      color='KGISDist 1', color_discrete_sequence=['darkblue']
#                      )   # Include district names in labels

#     # Customize the layout to make text labels visible
#     fig.update_traces(textposition='top center', showlegend=False)
#     fig.update_layout(annotations=[
#         dict(
#             x=0.5,
#             y=-0.25,
#             showarrow=False,
#             text=f'Alpha 1: {alpha1}, Alpha 2: {alpha2}, Alpha 3: {alpha3}, Alpha 4: {alpha4}',
#             xref="paper",
#             yref="paper"
#         )])

#     # Add average lines
#     avg_stability = filtered_df['New Stability'].mean()
#     avg_impact = filtered_df['Impact'].mean()

#     fig.add_shape(
#         type='line',
#         x0=avg_stability, y0=0, x1=avg_stability, y1=1,
#         line=dict(color='red', width=2),
#         yref='paper'
#     )

#     fig.add_shape(
#         type='line',
#         x0=0, y0=avg_impact, x1=1, y1=avg_impact,
#         line=dict(color='red', width=2),
#         xref='paper'
#     )

#     # Add text annotations
#     fig.add_trace(go.Scatter(
#         x=[avg_stability],
#         y=[0],
#         text=[f'Avg Stability: {avg_stability:.2f}'],
#         mode="text",
#     ))

#     fig.add_trace(go.Scatter(
#         x=[0.1],
#         y=[avg_impact-0.07],
#         text=[f'Avg Impact: {avg_impact:.2f}'],
#         mode="text",
#     ))

#     # Clear the output cell to remove the old plot
#     clear_output()
#     # Display the new plot
#     display(fig)

# # Create sliders for alphas
# alpha1_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 1:')
# alpha2_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 2:')
# alpha3_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 3:')
# alpha4_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 4:')

# # Create button to update plot
# update_button = widgets.Button(description="Update Plot")
# update_button.on_click(lambda _: update_plot(alpha1_slider.value, alpha2_slider.value, alpha3_slider.value, alpha4_slider.value))

# # Organize sliders and button in a vertical box
# controls_box = widgets.VBox([alpha1_slider, alpha2_slider, alpha3_slider, alpha4_slider, update_button])

# # Display the initial plot
# update_plot(alpha1_slider.value, alpha2_slider.value, alpha3_slider.value, alpha4_slider.value)

# # Display the sliders and button below the plot
# widgets.VBox([controls_box])


In [111]:
# import pandas as pd

# def convert_xlsx_to_csv(xlsx_file, csv_file):
#     # Read the Excel file into a pandas DataFrame
#     df = pd.read_excel(xlsx_file)

#     # Save the DataFrame to a CSV file
#     df.to_csv(csv_file, index=False)

# # Replace 'input_file.xlsx' with the path to your Excel file and 'output_file.csv' with the desired CSV file name
# xlsx_file_path = 'x_col_new_all_comb_5.xlsx'
# csv_file_path = 'x_col_new_all_comb_5_c.csv'

# convert_xlsx_to_csv(xlsx_file_path, csv_file_path)